# 데이터 스트림 압축기 – 임계값 버전 (Adaptive Stream Compressor – Threshold)

**TAG:** Greedy, Sliding Window, Monotonic Queue (Deque)

---

## 📄 문제 설명

정수 배열 `nums`와 정수 `K`, 그리고 **임계값** `T`가 주어집니다. 배열을 **여러 개의 연속 구간**으로 분할하려고 합니다. 각 구간 `[i, j]`(포함)의 **압축 다양성 비용**을

```
var(i, j) = max(nums[i..j]) - min(nums[i..j])
```

이라고 할 때, 각 구간에 고정 오버헤드 `K`가 추가로 든다고 하자. 단, **구간이 유효하려면** `var(i, j) + K ≤ T` 를 만족해야 합니다.

배열 전체를 유효한 구간들로 나누되, **필요한 구간의 최소 개수**를 구하세요. (같은 의미로, 전체 비용이 `segments * K + ∑var` 라면, 모든 유효 구간은 상한 `T` 내이므로 **구간 수를 최소화**하는 것이 목표와 동치입니다.)

> 직관: 한 구간의 다양성(`max-min`)과 오버헤드 `K`의 합이 `T`를 넘지 않는 한, 가능한 한 **길게** 구간을 확장하는 것이 전체 구간 수를 줄이는 최선입니다.

---

## ⛔ 제약 조건

* `1 ≤ N = len(nums) ≤ 10^5`
* `1 ≤ nums[i] ≤ 10^9`
* `0 ≤ K ≤ 10^9`, `0 ≤ T ≤ 10^9`
* 항상 적어도 하나의 해가 존재한다고 가정하지 않습니다. **해가 없으면 `-1`을 반환**합니다.

---

## 🧠 예시

### 예시 1

```
입력: nums = [1, 5, 6, 2, 3], K = 2, T = 7
출력: 1
설명: 전체 하나의 구간 [0..4] 에 대해 var = 6-1 = 5, 5 + K = 7 ≤ T 이므로 1개 구간이면 충분.
```

### 예시 2

```
입력: nums = [4, 3, 1, 10, 5, 8], K = 3, T = 8
가능한 출력(1단계 기준): 2
가능한 분할(2단계 기준의 한 예): [0..2], [3..5]
설명: 각 구간은 var + K ≤ 8을 만족해야 함. 그리디하게 확장하다가 조건을 깨면 직전에서 구간을 끊음.
```

> 인덱스는 **0-기반, 구간은 양끝 포함**(`[L..R]`)입니다.

---

## 💡 권장 접근 (힌트)

* **그리디 + 슬라이딩 윈도우**: 왼쪽 포인터 `L`, 오른쪽 포인터 `R`로 윈도우를 확장합니다.
* 윈도우 내 **최대/최소**를 `deque` 두 개(단조 감소/증가)로 유지하면 `O(1)`로 `max-min`을 구할 수 있습니다.
* 조건 `max - min + K ≤ T`를 만족하는 동안 `R`을 늘리고, 조건을 처음으로 **넘어서는 순간 직전에서 구간을 확정**합니다.
* **불가능 판정**: 단일 원소의 다양성은 `0`이므로, `K > T`이면 어떤 구간도 만들 수 없습니다(즉시 `-1`).

**시간 복잡도:** `O(N)` (각 원소가 덱에 들어갔다 나오는 횟수는 최대 1회)
**공간 복잡도:** `O(N)` (최악의 경우 덱에 단조로 원소가 쌓일 수 있음)

---

# ✅ 단계별 구현 과제

두 단계로 나누어 구현하십시오. **각 단계는 독립적으로 채점**됩니다.

## 1단계 (필수) — 최소 구간 개수만 반환

아래 함수를 구현하세요. **최소 구간 수**만 반환합니다. 해가 없으면 `-1`을 반환합니다.

```python
from typing import List

class Solution:
    def minSegmentsUnderThreshold(self, nums: List[int], K: int, T: int) -> int:
        """배열을 var+K ≤ T 인 연속 구간들로 분할할 때 필요한 최소 구간 수.
        불가능하면 -1을 반환.
        """
        pass
```

### 1단계 채점 포인트

* 그리디 확장 후 **조건 위반 시 직전에서 즉시 자르기**
* `K > T`인 경우 **즉시 -1**
* 시간/공간 복잡도 준수 (`O(N)`/`O(N)`)

### 1단계 예제 입출력(참고)

* `([1,5,6,2,3], K=2, T=7) → 1`
* `([4,3,1,10,5,8], K=3, T=8) → 2`
* `([2,2,2], K=1, T=1) → 1`
* `([2,2,2], K=2, T=1) → -1`

---

## 2단계 (확장) — 최소 구간 개수와 **실제 분할 구간**까지 반환

아래 확장 함수를 구현하세요. 2단계에서는 **최소 구간 수와 그 예시 분할(구간 리스트)**를 함께 반환합니다. 여러 최적 해가 있으면, **좌→우 그리디**로 얻는 분할을 반환하면 충분합니다.

```python
from typing import List, Tuple

class Solution:
    def minSegmentsUnderThresholdWithCuts(self, nums: List[int], K: int, T: int) -> Tuple[int, List[Tuple[int, int]]]:
        """최소 구간 수와, 그에 대응하는 [L, R] (0-기반, 양끝 포함) 분할을 반환.
        불가능하면 (-1, []).
        """
        pass
```

### 2단계 출력 형식

* 반환값: `(segments, cuts)`

  * `segments`: 최소 구간 개수 (정수)
  * `cuts`: `[(L1, R1), (L2, R2), ...]` 형태의 구간 리스트 (각 구간은 **연속**, **겹치지 않음**, **전 구간을 정확히 덮음**)
* 불가능한 경우: `(-1, [])`

### 2단계 채점 포인트

* 1단계 로직을 재사용하되 **구간 경계 기록**
* **정확한 커버리지**: `cuts`가 `[0..n-1]`을 정확히 덮어야 함
* `K > T`인 경우 **(-1, [])**

---

## 🧠 학습 포인트 정리

* **그리디**: 가능한 한 긴 유효 구간을 만들어 전체 구간 수를 최소화
* **슬라이딩 윈도우 + 단조 큐**: 윈도우 내 `max/min`을 `O(1)`에 가깝게 유지
* **불가능 판정**: 단일 원소조차 허용되지 않으면 전체도 불가능

> 고급 연결: 본 변형은 임계값 제약 하에서의 **분할 개수 최소화** 문제입니다. 원문(난이도 높음)에서의 비용 직접 최소화(DP/최적화)와 대비해, 본 문제는 **그리디가 최적**임을 관찰하고 효율적으로 구현하는 데 초점을 둡니다.


In [ ]:
from collections import deque
from typing import List, Tuple

class Solution:
    # 1단계: 최소 구간 수만 반환
    def minSegmentsUnderThreshold(self, nums: List[int], K: int, T: int) -> int:
        """배열을 var+K ≤ T 인 연속 구간들로 분할할 때 필요한 최소 구간 수.
        불가능하면 -1을 반환.
        규약: 0-기반 인덱스, 구간은 양끝 포함. 빈 배열은 0 반환.
        """
        # TODO: 구현
        pass

    # 2단계: 최소 구간 수 + 실제 [L, R] 분할까지 반환
    def minSegmentsUnderThresholdWithCuts(self, nums: List[int], K: int, T: int) -> Tuple[int, List[Tuple[int, int]]]:
        """(segments, cuts) 반환. 불가능하면 (-1, []).
        cuts는 [(L1,R1), (L2,R2), ...]로 전체 [0..n-1]을 정확히 덮어야 함.
        빈 배열은 (0, []).
        """
        # TODO: 구현
        pass


# Test
def run_tests():
    sol = Solution()

    # 케이스 1
    nums = [1, 5, 6, 2, 3]; K = 2; T = 7
    print("[1단계] 최소 구간 수만")
    print(sol.minSegmentsUnderThreshold(nums, K, T))  # 1

    print("\n[2단계] 최소 구간 수 + 구간들")
    segs, cuts = sol.minSegmentsUnderThresholdWithCuts(nums, K, T)
    print(segs, cuts)  # 1 [(0,4)]

    # 케이스 2
    nums2 = [4, 3, 1, 10, 5, 8]; K2 = 3; T2 = 8
    print("\n[1단계] 최소 구간 수만 (케이스2)")
    print(sol.minSegmentsUnderThreshold(nums2, K2, T2))  # 2

    print("\n[2단계] 최소 구간 수 + 구간들 (케이스2)")
    segs2, cuts2 = sol.minSegmentsUnderThresholdWithCuts(nums2, K2, T2)
    print(segs2, cuts2)  # 2 예: [(0,2),(3,5)]

    # 불가능 케이스 (K > T)
    nums3 = [2,2,2]; K3 = 2; T3 = 1
    print("\n[불가능 케이스]")
    print(sol.minSegmentsUnderThreshold(nums3, K3, T3))  # -1
    print(sol.minSegmentsUnderThresholdWithCuts(nums3, K3, T3))  # (-1, [])

    # 시작=끝이 존재하는 최소 배열
    nums4 = [1]; K4 = 0; T4 = 0
    print("\n[단일 원소]")
    print(sol.minSegmentsUnderThreshold(nums4, K4, T4))  # 1
    print(sol.minSegmentsUnderThresholdWithCuts(nums4, K4, T4))  # (1, [(0,0)])

    # 빈 배열(선택적 처리)
    nums5 = []
    print("\n[빈 배열]")
    print(sol.minSegmentsUnderThreshold(nums5, 1, 5))   # 0
    print(sol.minSegmentsUnderThresholdWithCuts(nums5, 1, 5))  # (0, [])

# Colab에서는 아래 한 줄만 실행하면 됩니다.
run_tests()
